In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import torch

In [ ]:
res = pd.read_csv('results/experiment_results_all.csv')

In [ ]:
res = res[2:6]
res["rmse"]

In [ ]:
fig = go.Figure()

fig.update_layout(title = '',
                   xaxis_title = 'grid cell size in meters',
                   yaxis_title = 'RMSE in meters',
                   width = 400,
                   height = 300)

fig.add_trace(go.Scatter(x = res["resolution"], y = res["rmse"], 
                    mode = 'lines+markers',
                    marker = dict(color = "dodgerblue"),
                    name = 'Antarctica (Bedmap3)',
                    showlegend = False))

fig.add_vline(x = 500, line_width = 3, line_dash = "dash", line_color = "grey")

fig.update_yaxes(range = [15, 57])
fig.update_xaxes(range = [110, 1010])

fig.update_layout(xaxis = dict(
        tickmode = 'array',
        tickvals = res["resolution"]))

fig.update_layout(template = "simple_white")
fig.update_layout(font_family = "Lato")
fig.update_xaxes(autorange = "reversed")

fig.update_xaxes(mirror = True, ticks = 'outside', showline = True)
fig.update_yaxes(mirror = True, ticks = 'outside', showline = True)

fig.show()

In [76]:
x_train = torch.tensor([0, 2, 3, 4, 7, 10, 12])
x_test = torch.arange(start = 0, end = 16, step = 1)

y_train = torch.tensor([10., 11., 14., 13., 7., 5., 6.])
l = 2.0
var_out = 1.0
var_noise = 0.5

K_xx = torch.exp(- torch.sub(torch.pow(x_train.unsqueeze(1) - x_train.unsqueeze(0), exponent = 2), (2*(l**2)))) * var_out
Kxstartx = torch.exp(- torch.sub(torch.pow(x_test.unsqueeze(1) - x_train.unsqueeze(0), exponent = 2), (2*(l**2)))) * var_out
K_xstartxstar = torch.exp(- torch.sub(torch.pow(x_test.unsqueeze(1) - x_test.unsqueeze(0), exponent = 2), (2*(l**2)))) * var_out

K_term = torch.inverse(K_xx * torch.eye(n = x_train.shape[0]) * var_noise)


y_preds = torch.matmul(torch.matmul(Kxstartx, K_term), y_train)

y_preds_covar = torch.sub(K_xstartxstar, torch.matmul(torch.matmul(Kxstartx, K_term), Kxstartx.mT))

In [77]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = x_test, y = y_preds, 
                    mode = 'lines+markers',
                    name = 'Mean predictions'))
fig.add_trace(go.Scatter(x = x_train, y = y_train, 
                    mode = 'lines+markers',
                    name = 'Data'))

In [78]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = x_test, y = y_preds, 
                    mode = 'lines+markers',
                    name = 'Mean predictions'))

for i in range(10):
    # Sample via Cholesky
    L = torch.linalg.cholesky(y_preds_covar)
    Z = torch.randn(size = [L.shape[-1]], dtype = torch.double).unsqueeze(0)
    sample = y_preds.double() + torch.matmul(Z.double(), L.double())  
    fig.add_trace(go.Scatter(x = x_test, y = sample.squeeze(), 
                        mode = 'lines',
                        name = 'Sample',
                        showlegend = False,
                        line = dict(
                                color='#0061ff',
                                width = 0.5
                            )
                        ))

fig.update_layout(title = '',
                   xaxis_title = 'coordinate',
                   yaxis_title = 'elevation')

fig.show()

_LinAlgError: linalg.cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 1 is not positive-definite).